# Run inference on time to merge model trained previously


## What we did previously

In the previous [notebook](./03_model_training.ipynb) we trained machine learning models to classify a PR's `time_to_merge` into one of the 10 bins (or "classes"). We then deployed the model with the highest f1-score as a service using the model saved in s3.

## In this step

In this notebook, we are going to fetch the model that we previously trained, saved and stored in s3. We will send a payload to this model and see how it performs on the test data.
# Time to Merge Prediction Inference Service

In the previous notebook, we explored some basic machine learning models for predicting time to merge of a PR.

In [1]:
import os
import sys
import gzip
import json
import boto3
import datetime
import requests
from dotenv import load_dotenv, find_dotenv
from io import BytesIO

import joblib

import numpy as np
import pandas as pd

from sklearn.metrics import classification_report

load_dotenv(find_dotenv(), override=True)

import warnings
warnings.filterwarnings('ignore') 

from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

In [2]:
class CephCommunication:
    """
    Class to establish communication with a ceph s3 bucket.
    It connects with the bucket and provides methods to read and write data in the parquet format.
    """

    def __init__(
        self, s3_endpoint_url, aws_access_key_id, aws_secret_access_key, s3_bucket
    ):
        self.s3_endpoint_url = s3_endpoint_url
        self.aws_access_key_id = aws_access_key_id
        self.aws_secret_access_key = aws_secret_access_key
        self.s3_resource = boto3.resource(
            "s3",
            endpoint_url=self.s3_endpoint_url,
            aws_access_key_id=self.aws_access_key_id,
            aws_secret_access_key=self.aws_secret_access_key,
        )
        self.bucket = s3_bucket
        ## Todo: Add try catch

    def upload_to_ceph(self, dataframe, s3_path, filename):
        """
        This helper function takes as input the data frame to be uploaded, and the output filename.
        It then saves the data frame in the defined ceph bucket.
        """
        parquet_buffer = BytesIO()
        dataframe.to_parquet(parquet_buffer)
        s3_obj = self.s3_resource.Object(self.bucket, f"{s3_path}/{filename}")
        status = s3_obj.put(Body=parquet_buffer.getvalue())
        return status

    def read_from_ceph(self, s3_path, filename):
        """
        Helper function to read from ceph and see if the saved data is correct.
        """
        buffer = BytesIO()
        s3_object = self.s3_resource.Object(self.bucket, f"{s3_path}/{filename}")
        s3_object.download_fileobj(buffer)
        df_temp = pd.read_parquet(buffer)
        return df_temp


def save_to_disk(dataframe, path, filename):
    """
    Helper function to save the dataframe
    as a parquet file to disk.
    """
    dataset_base_path = Path(path)
    dataset_base_path.mkdir(parents=True, exist_ok=True)
    dataframe.to_parquet(f"{path}/{filename}")
    return True

In [3]:
## CEPH Bucket variables
## Create a .env file on your local with the correct configs,

ORG = os.getenv("GITHUB_ORG")
REPO = os.getenv("GITHUB_REPO")

## S3 bucket credentials
s3_endpoint_url = os.getenv("S3_ENDPOINT_URL")
s3_access_key = os.getenv("AWS_ACCESS_KEY_ID")
s3_secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")
s3_bucket = os.getenv("S3_BUCKET")

s3_input_data_path = os.getenv("CEPH_BUCKET_PREFIX")

REMOTE = os.getenv("REMOTE")
RAW_DATA_PATH = os.path.join(
    s3_input_data_path, "srcopsmetrics/bot_knowledge", ORG, REPO, "PullRequest.json"
)

In [4]:
output = []
local_input_data_path = "../../../data/raw/GitHub/PullRequest.json.gz"
if REMOTE:
    print("getting dataset from ceph")
    s3 = boto3.resource(
        "s3",
        endpoint_url=s3_endpoint_url,
        aws_access_key_id=s3_access_key,
        aws_secret_access_key=s3_secret_key,
    )
    content = s3.Object(s3_bucket, RAW_DATA_PATH)
    file = content.get()["Body"].read().decode("utf-8")

    prs = json.loads(file)

    for pr in prs.splitlines():
        output.append(json.loads(pr))

else:
    print("getting dataset from local")
    with gzip.open(local_input_data_path, "r") as f:
        prs = json.loads(f.read().decode("utf-8"))


pr_df = pd.DataFrame(output)

getting dataset from ceph


In [5]:
# github pr dataset collected using thoth's mi-scheduler
pr_df.head()

,title,body,size,created_by,created_at,closed_at,closed_by,merged_at,merged_by,commits_number,changed_files_number,interactions,reviews,labels,commits,changed_files,first_review_at,first_approve_at,id
0,Update the repository for creating access issues,Signed-off-by: Shrey <shanand@redhat.com>,XS,Shreyanand,1667406781,NaN,None,NaN,None,1,1,{'sesheta': 86},{},[size/XS],[3b58d0c6b2dd9aacab6ada9df6502c8aecbb0702],[docs/access-jh.md],NaN,NaN,227
1,Changed the environment variable name to origi...,[Here](https://github.com/os-climate/aicoe-osc...,M,suppathak,1667228783,1.667242e+09,sesheta,1.667242e+09,sesheta,1,5,"{'review-notebook-app[bot]': 29, 'suppathak': ...","{'1162135879': {'author': 'Shreyanand', 'words...","[approved, lgtm, size/M]",[fa5ea4fe2f9db12c167d7d4ec18651c178efc2d5],"[notebooks/demo2/create_results_table.ipynb, n...",1.667232e+09,NaN,225
2,Adjusted envorinment variable names,Closes #218,XXL,suppathak,1666738188,1.666872e+09,sesheta,1.666872e+09,sesheta,1,5,"{'review-notebook-app[bot]': 29, 'erikerlandso...","{'1156963848': {'author': 'Shreyanand', 'words...","[size/XXL, approved, lgtm]",[f22b079bd535183fe2a29143c873b6a54e782194],"[notebooks/demo2/create_results_table.ipynb, n...",1.666804e+09,NaN,219
3,Add time analysis notebook and dataframe,This PR adds benchmark run data and analysis n...,XXL,Shreyanand,1666135746,NaN,None,NaN,None,1,3,"{'review-notebook-app[bot]': 29, 'sesheta': 86}",{},[size/XXL],[cd9b9f82045e76c2484a6a8f3342915efc6384a1],"[notebooks/demo2/time-anlysis.ipynb, reports/b...",NaN,NaN,217
4,Add transformer implementation of relevance task,This PR adds transformer implementation and di...,XXL,Shreyanand,1665611915,NaN,None,NaN,None,1,3,"{'review-notebook-app[bot]': 29, 'sesheta': 86...","{'1162026715': {'author': 'suppathak', 'words_...",[size/XXL],[29077cb9934f76a1ec489999bbd9699bc3acbb74],"[notebooks/demo2/transformer-relevance.ipynb, ...",1.667228e+09,NaN,216


In [6]:
interval = (pr_df["merged_at"] - pr_df["created_at"]).astype("float")
interval = interval.dropna()
interval

1       13666.0
2      133369.0
6      264782.0
8     1826092.0
9     1723514.0
        ...    
88        680.0
89     150846.0
90      94769.0
91      64366.0
92      62135.0
Length: 77, dtype: float64

In [7]:
n_buckets = 10

quantiles = interval.quantile(q=np.arange(0, 1 + 1e-100, 1 / n_buckets))
quantiles

0.0        149.0
0.1       1235.0
0.2       4947.8
0.3      41032.2
0.4      74179.0
0.5     133369.0
0.6     236995.6
0.7     345272.8
0.8     608485.6
0.9    1407709.4
dtype: float64

In [8]:
quantiles / 3600

0.0      0.041389
0.1      0.343056
0.2      1.374389
0.3     11.397833
0.4     20.605278
0.5     37.046944
0.6     65.832111
0.7     95.909111
0.8    169.023778
0.9    391.030389
dtype: float64

In [9]:
time_intervals = quantiles / 3600

In [10]:
# remove PRs from train/test which are still open
pr_df = pr_df[pr_df["closed_at"].notna()]
pr_df = pr_df[pr_df["merged_at"].notna()]

In [11]:
pr_df["created_at"] = pr_df["created_at"].apply(
    lambda x: int(datetime.datetime.timestamp(pd.to_datetime(x)))
)
pr_df["closed_at"] = pr_df["closed_at"].apply(
    lambda x: float(datetime.datetime.timestamp(pd.to_datetime(x)))
)
pr_df["merged_at"] = pr_df["merged_at"].apply(
    lambda x: float(datetime.datetime.timestamp(pd.to_datetime(x)))
)

In [12]:
TEST_DATA_PATH = os.path.join(s3_input_data_path, ORG, REPO, "test-data")

# read processed and split data created for train/test in the model training notebook
if REMOTE:
    cc = CephCommunication(s3_endpoint_url, s3_access_key, s3_secret_key, s3_bucket)
    X_test = cc.read_from_ceph(TEST_DATA_PATH, "X_test.parquet")
    y_test = cc.read_from_ceph(TEST_DATA_PATH, "y_test.parquet")

else:
    print(
        "The X_test.parquet and y_test.parquet files are not included in the github repo."
    )
    print(
        "Please set REMOTE=1 in the .env file and read this data from the S3 bucket instead."
    )

In [13]:
X_test

,size,created_at_day,created_at_month,created_at_weekday,created_at_hour,changed_files_number,body_size,commits_number,filetype_.py,filetype_.ipynb,...,title_wordcount_version,title_wordcount_versions,title_wordcount_via,title_wordcount_video,title_wordcount_w,title_wordcount_wip,title_wordcount_work,title_wordcount_workflow,title_wordcount_write,title_wordcount_yaml
9,4.0,25.0,8.0,3.0,20.0,1.0,3.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49,3.0,18.0,11.0,3.0,18.0,2.0,36.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15,0.0,1.0,8.0,0.0,21.0,2.0,25.0,1.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.0,31.0,10.0,0.0,15.0,5.0,40.0,1.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60,0.0,29.0,10.0,4.0,20.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
63,5.0,28.0,10.0,3.0,22.0,8.0,43.0,3.0,6.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
82,3.0,6.0,10.0,2.0,12.0,2.0,19.0,1.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
69,2.0,21.0,10.0,3.0,15.0,2.0,55.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
66,0.0,25.0,10.0,0.0,17.0,1.0,17.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
41,0.0,14.0,12.0,1.0,20.0,1.0,23.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
y_test

,ttm_class
9,9
49,9
15,5
1,2
60,0
63,6
82,2
69,6
66,1
41,3


In [15]:
sample_payload = pr_df.reindex(X_test.index)

In [16]:
sample_payload

,title,body,size,created_by,created_at,closed_at,closed_by,merged_at,merged_by,commits_number,changed_files_number,interactions,reviews,labels,commits,changed_files,first_review_at,first_approve_at,id
9,Move results table to Iceberg catalog on cl2,Related issue #169,XL,Shreyanand,1,1.663182,sesheta,1.663182,sesheta,1,1,"{'review-notebook-app[bot]': 29, 'MichaelCliff...",{},"[size/XL, approved, lgtm]",[5e650430e3f0ad60cc194a7f30755050f7e9cc07],[notebooks/demo2/create_results_table.ipynb],NaN,NaN,199
49,Add demo2 superset dashboard,"Relevant issues #105 \r\n\r\nIn this PR, I add...",L,Shreyanand,1,1.638973,MichaelClifford,1.638973,MichaelClifford,1,2,"{'aakankshaduggal': 82, 'Shreyanand': 85, 'Mic...",{},[],[190fbf1702cce11766e7258f0cc566c1676c0282],"[data/superset/demo1.json, data/superset/demo2...",NaN,NaN,107
15,Disable multiprocessing for kpi-training,Signed-off-by: Shreyanand <shanand@redhat.com>...,XS,Shreyanand,1,1.659619,sesheta,1.659619,sesheta,1,2,"{'MichaelClifford': 2, 'sesheta': 65}",{},"[approved, lgtm, size/XS]",[24bb3caef4baa6a357586de81952ffb4150684d1],"[notebooks/demo2/config_qa_farm_train.py, src/...",NaN,NaN,189
1,Changed the environment variable name to origi...,[Here](https://github.com/os-climate/aicoe-osc...,M,suppathak,1,1.667242,sesheta,1.667242,sesheta,1,5,"{'review-notebook-app[bot]': 29, 'suppathak': ...","{'1162135879': {'author': 'Shreyanand', 'words...","[approved, lgtm, size/M]",[fa5ea4fe2f9db12c167d7d4ec18651c178efc2d5],"[notebooks/demo2/create_results_table.ipynb, n...",1.667232e+09,NaN,225
60,added demo 1 video link,SSIA,XS,oindrillac,1,1.635538,MichaelClifford,1.635538,MichaelClifford,1,1,{'MichaelClifford': 5},{},[],[d01392edd381cd9b19392545e7961e7a7605bc76],[notebooks/demo1/README.md],NaN,NaN,77
63,Add training environment,This PR adds source code needed for running tr...,XXL,chauhankaranraj,1,1.635793,Shreyanand,1.635793,Shreyanand,3,8,"{'MichaelClifford': 1, 'chauhankaranraj': 48}","{'793217363': {'author': 'Shreyanand', 'words_...",[],"[d4434b1e046dfeca8fb6b6b40d5160ee79d468d1, 0f3...","[Pipfile, Pipfile.lock, src/components/utils/q...",1.635523e+09,1.635793e+09,74
82,Track kfp pipeline metrics for demo1,Introducing kfpipeline metrics in both steps o...,L,oindrillac,1,1.633547,MichaelClifford,1.633547,MichaelClifford,1,2,"{'review-notebook-app[bot]': 29, 'MichaelCliff...",{},[],[cc166def45ceaf1727952e8f71767d903d422744],"[notebooks/demo1-create-tables.ipynb, notebook...",NaN,NaN,35
69,Add Dockerfile for custom image,## Related issue\r\nFixes #45 \r\n\r\n## Descr...,M,Shreyanand,1,1.635175,chauhankaranraj,1.635175,chauhankaranraj,1,2,{'chauhankaranraj': 2},"{'786011071': {'author': 'chauhankaranraj', 'w...",[],[3f174075107d08f0a634f93b06f5cfa9fdd50360],"[.aicoe-ci.yaml, Dockerfile]",1.634838e+09,1.635174e+09,65
66,Add git repo environment variables to dockerfile,Update Dockerfile to include environment varia...,XS,chauhankaranraj,1,1.635183,chauhankaranraj,1.635183,chauhankaranraj,1,1,{'chauhankaranraj': 3},"{'788404127': {'author': 'harshad16', 'words_c...",[],[273a1f3dca27342584a8513cbd57f0d7c13a4789],[Dockerfile],1.635182e+09,1.635183e+09,70
41,Update README,This PR \r\n- updates the README to mention th...,XS,chauhankaranraj,1,1.639573,MichaelClifford,1.639573,MichaelClifford,1,1,{'MichaelClifford': 1},{},[],[ad9668f096e1e5ebc5b123d1c6df4ea5ed98af5a],[notebooks/demo2/README.md],NaN,NaN,124


In [17]:
sample_payload.dtypes

title                    object
body                     object
size                     object
created_by               object
created_at                int64
closed_at               float64
closed_by                object
merged_at               float64
merged_by                object
commits_number            int64
changed_files_number      int64
interactions             object
reviews                  object
labels                   object
commits                  object
changed_files            object
first_review_at         float64
first_approve_at        float64
id                       object
dtype: object

In [18]:
## read model
MODEL_KEY = os.path.join(s3_input_data_path, ORG, REPO, "ttm-model")
MODEL_FILENAME = "model.joblib"


s3_resource = boto3.resource(
    "s3",
    endpoint_url=s3_endpoint_url,
    aws_access_key_id=s3_access_key,
    aws_secret_access_key=s3_secret_key,
)

buffer = BytesIO()
s3_object = s3_resource.Object(s3_bucket, f"{MODEL_KEY}/{MODEL_FILENAME}")
s3_object.download_fileobj(buffer)
model = joblib.load(buffer)
model

Pipeline(steps=[('extract_features',
                 ColumnTransformer(transformers=[('pr_size',
                                                  OrdinalEncoder(categories=[['XS',
                                                                              'S',
                                                                              'M',
                                                                              'L',
                                                                              'XL',
                                                                              'XXL']]),
                                                  ['size']),
                                                 ('created_at_details',
                                                  DateTimeDetailsTransformer(),
                                                  ['created_at']),
                                                 ('n_changed_files',
                                                  NumChangedFilesTransformer(),
                                                  ['changed_files_number']),
                                                 ('body_size',
                                                  StringLenTransformer(),
                                                  ['body']),
                                                 ('n_commits', 'p...
                               gamma=0, gpu_id=-1, grow_policy='depthwise',
                               importance_type=None, interaction_constraints='',
                               learning_rate=0.1, max_bin=256,
                               max_cat_to_onehot=4, max_delta_step=0,
                               max_depth=6, max_leaves=0, min_child_weight=1,
                               missing=nan, monotone_constraints='()',
                               n_estimators=125, n_jobs=-1, num_parallel_tree=1,
                               objective='multi:softprob', predictor='auto',
                               random_state=42, reg_alpha=0, ...))])

In [19]:
# Test model on the dataset
preds = model.predict(sample_payload)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.12      1.00      0.22         1
           1       0.00      0.00      0.00         3
           2       0.00      0.00      0.00         2
           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00         1
           5       0.00      0.00      0.00         1
           6       0.00      0.00      0.00         2
           7       0.00      0.00      0.00         1
           9       0.17      0.33      0.22         3

    accuracy                           0.12        16
   macro avg       0.03      0.15      0.05        16
weighted avg       0.04      0.12      0.06        16



# Conclusion

This notebook fetches the saved model from s3 and sends a payload to see how the model is performing on this new data. Additionally, we see that the evaluation scores in the classification report match the ones we saw in the training notebook. So, great, looks like our model are working as expected, and are ready to predict some times to merge for GitHub PRs! 